In [3]:
import pandas as pd
import numpy as np
import json
import os

outdir = "./traffic/"
if not os.path.exists(outdir):
    os.makedirs(outdir)
# Load your datasets
train_data = pd.read_csv('loop_sensor_train.csv')
metadata = pd.read_csv('geo_reference.csv', delimiter=';')


/tmp/ipykernel_851702/2829014297.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_data['type'] = 'Point'


Files created: traffic.geo, traffic.dyna, traffic.rel, config.json


In [4]:
'''
iu_ac        -> geo_id
geo_point_2d -> coordinates
'''
# Create .geo file
geo_data = metadata[['iu_ac', 'geo_point_2d']]
geo_data.columns = ['geo_id', 'coordinates']
geo_data['type'] = 'Point'
geo_data['coordinates'] ='[' + geo_data['coordinates'] +']'
geo_data = geo_data[['geo_id', 'type', 'coordinates']]
geo_data.to_csv(outdir+'traffic.geo', index=False)


/tmp/ipykernel_851702/2621750846.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_data['type'] = 'Point'
/tmp/ipykernel_851702/2621750846.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_data['coordinates'] ='[' + geo_data['coordinates'] +']'


In [ ]:

# Create .dyna file
train_data['dyna_id'] = range(len(train_data))
dyna_data = train_data[['dyna_id', 't_1h', 'iu_ac', 'q']]
dyna_data.columns = ['dyna_id', 'time', 'entity_id', 'traffic_flow']
dyna_data.to_csv(outdir+'traffic.dyna', index=False)


In [ ]:

# Create .rel file - assume a simplistic relation, perhaps distance, which needs to be added if it doesn't exist
# This part is hypothetical and assumes a relationship based on some distance metric
rel_data = metadata[['iu_ac']].drop_duplicates()
rel_data['rel_id'] = range(len(rel_data))
rel_data['distance'] = np.random.uniform(1, 100, len(rel_data))  # Random distances as an example
rel_data['type'] = 'geo'
rel_data.columns = ['entity_id', 'rel_id', 'distance', 'type']
rel_data = rel_data[['rel_id', 'type', 'entity_id', 'distance']]
rel_data.to_csv(outdir+'traffic.rel', index=False)


In [ ]:

# Create config.json file
config_data = {
    "geo": {
        "including_types": ["Point"],
        "Point": {}
    },
    "rel": {
        "including_types": ["geo"],
        "geo": {
            "distance": "num"
        }
    },
    "dyna": {
        "including_types": ["state"],
        "state": {
            "entity_id": "geo_id",
            "traffic_flow": "num"
        }
    },
    "info": {
        "data_col": ["traffic_flow"],
        "weight_col": "distance",
        "data_files": ["traffic"],
        "geo_file": "traffic",
        "rel_file": "traffic",
        "output_dim": 1,
        "time_intervals": 300,
        "init_weight_inf_or_zero": "zero",
        "set_weight_link_or_dist": "link",
        "calculate_weight_adj": False,
        "weight_adj_epsilon": 0.1
    }
}

with open(outdir+'config.json', 'w') as fp:
    json.dump(config_data, fp, indent=4)

print("Files created: traffic.geo, traffic.dyna, traffic.rel, config.json")
